In [ ]:
# model
import torch
import torch.nn as nn

# data 
import torchtext
from torchtext.datasets import Multi30k
from torchtext.data import Field, BucketIterator

#training 
import tqdm 

In [ ]:
### Random seed for deterministic results
SEED = 42
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [4]:
device

device(type='cuda', index=0)

### Data 

In [5]:
BATCH_SIZE = 128
max_len = 48

In [6]:
# SRC Field
SRC = Field(
    init_token="<sos>",
    eos_token="<eos>",
    tokenize="spacy",
    tokenizer_language="de",
    batch_first=True,
    lower=True
)

In [7]:
# TRG Field
TRG = Field(
    init_token="<sos>",
    eos_token="<eos>",
    tokenize="spacy",
    tokenizer_language="en",
    batch_first=True,
    lower=True
)

In [8]:
## We're using Multi30K dataset
train, valid, test = Multi30k.splits(exts=(".de", ".en"), fields=(SRC, TRG))

In [9]:
# Bulding the vocab
SRC.build_vocab(train, min_freq=2)
TRG.build_vocab(train, min_freq=2)

In [10]:
# Data loader
train_loader, val_loader, test_loader = BucketIterator.splits(
    datasets=(train, valid, test),
    batch_size=BATCH_SIZE,
    device=device
)

In [11]:
for i, batch in enumerate(train_loader):
    print(batch.src.size())
    print(batch.trg.size())
    break

torch.Size([128, 29])
torch.Size([128, 30])


In [12]:
## Vocan Insepction
print(f'English vocab {len(TRG.vocab)}\nGerman vocab {len(SRC.vocab)}')

English vocab 5893
German vocab 7853


### Seq2Seq Model

#### Encoder

In [13]:
class Encoder(nn.Module):
    
    def __init__(self, hidden_dim, embedding_dim, vocab_size, num_layers=2, dropout=0.25):
        super(Encoder, self).__init__()
        
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        
        self.embedding = nn.Embedding(num_embeddings=vocab_size, embedding_dim=embedding_dim)
        self.seq = nn.LSTM(
            input_size=embedding_dim, 
            hidden_size=hidden_dim, 
            num_layers=num_layers, 
            batch_first=True, 
            dropout=dropout
        )
        self.dropout = nn.Dropout(p=dropout)
    
    def forward(self, src):
        
        embedded = self.embedding(src)
        
        outputs, (hidden, cell) = self.seq(embedded)
        
        return hidden, cell        

#### Decoder

In [14]:
class Decoder(nn.Module):
    
    def __init__(self, vocab_size, embedding_dim, hidden_dim, num_layers=2, dropout=0.25):
        super(Decoder, self).__init__()
        
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.output_dim = vocab_size
        
        self.embedding  = nn.Embedding(num_embeddings=vocab_size, embedding_dim=embedding_dim)
        self.seq = nn.LSTM(
            input_size=embedding_dim, 
            hidden_size=hidden_dim, 
            num_layers=num_layers, 
            batch_first=True, 
            dropout=dropout
        )
        self.dropout = nn.Dropout(p=dropout)
        self.fc = nn.Linear(in_features=hidden_dim, out_features=vocab_size)
    
    def forward(self, x, hidden, cell):
        
#         x = x.unsqueeze(1)
        
        embedded = self.dropout(self.embedding(x))
        
        output, (hidden, cell) = self.seq(embedded, (hidden, cell))
        
        prediction = self.fc(output.squeeze(0))
        
        return prediction, hidden, cell    

In [15]:
## Hidden_dim dim and num_layers of encoder and decoder must be sam 

In [17]:
class Seq2Seq(nn.Module):
    
    def __init__(self, encoder, decoder):
        super(Seq2Seq, self).__init__()
        self.encoder  = encoder
        self.decoder = decoder
        
    
    def forward(self, src, trg, teacher_forcing_ratio = 0.5):
        
        batch_size = src.size(0)
        trg_len = trg.size(1)
        
        trg_vocab_size = self.decoder.output_dim
        
        hidden, cell = encoder(src)
        
        prediction, _, _ = decoder(trg, hidden, cell)
        
        return prediction 
             

### Training

In [18]:
import torch.utils.tensorboard as tensorboard

In [19]:
encoder = Encoder(hidden_dim=64, embedding_dim=100, vocab_size=len(SRC.vocab)).to(device)
decoder = Decoder(vocab_size=len(TRG.vocab), embedding_dim=100, hidden_dim=64).to(device)
model = Seq2Seq(encoder, decoder).to(device)

In [20]:
lr = 1e-3
epochs = 20
TRG_PAD_IDX = TRG.vocab.stoi[TRG.pad_token]
writer = tensorboard.SummaryWriter()

In [21]:
###
optimizer = torch.optim.Adam(params=model.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss(ignore_index=TRG_PAD_IDX).to(device)

In [22]:
def eval(model, data):
    losses = []
    with torch.no_grad():
        for batch in data:
            outputs = model(batch.src, batch.trg)
            loss = criterion(outputs.view(outputs.shape[0]*outputs.shape[1], outputs.shape[2]), batch.trg.view(-1))
            losses.append(loss.item())
    return sum(losses)/len(losses)
            

In [23]:
steps = 0
epoch_progress = tqdm.tqdm(total=epochs, desc="Epoch", position=0)

for epoch in range(epochs):
    
    step_progress = tqdm.tqdm(total=len(train_loader), )
    train_loss = []
    
    for batch in train_loader:
        src = batch.src
        trg = batch.trg
        
        outputs = model(src, trg)
        
        loss = criterion(outputs.view(outputs.shape[0]*outputs.shape[1], outputs.shape[2]), trg.view(-1))
        loss.backward()
        optimizer.step()
        
        
        if steps%100==0:
            print(f'Epoch {epoch} | Step {steps} | Train_loss {loss.item()} | Val_loss = {eval(model, val_loader)}')
            
        steps += 1
        step_progress.update(1)
    epoch_progress.update(1)    

  3%|▎         | 7/227 [00:00<00:18, 11.86it/s]

Epoch 0 | Step 0 | Train_loss 8.672889709472656 | Val_loss = 8.658742308616638



 48%|████▊     | 108/227 [00:01<00:02, 58.52it/s]

Epoch 0 | Step 100 | Train_loss 6.064271450042725 | Val_loss = 5.928687572479248



 94%|█████████▍| 213/227 [00:03<00:00, 60.10it/s]

...

Epoch:  95%|█████████▌| 19/20 [01:14<00:04,  4.23s/it]







  0%|          | 0/227 [00:00<?, ?it/s]







  2%|▏         | 5/227 [00:00<00:04, 46.28it/s]







  5%|▍         | 11/227 [00:00<00:04, 48.34it/s]







  7%|▋         | 17/227 [00:00<00:04, 50.22it/s]







 10%|█         | 23/227 [00:00<00:03, 51.53it/s]







 13%|█▎        | 29/227 [00:00<00:03, 52.49it/s]







 15%|█▌        | 35/227 [00:00<00:03, 53.29it/s]







 18%|█▊        | 41/227 [00:00<00:03, 54.45it/s]







 21%|██        | 47/227 [00:00<00:03, 49.46it/s]







 23%|██▎       | 52/227 [00:01<00:03, 49.44it/s]







 26%|██▌       | 59/227 [00:01<00:03, 53.21it/s]







 29%|██▊       | 65/227 [00:01<00:03, 49.85it/s]







 31%|███▏      | 71/227 [00:01<00:02, 52.12it/s]







 34%|███▍      | 78/227 [00:01<00:02, 55.66it/s]







 37%|███▋      | 84/227 [00:01<00:02, 53.76it/s]







 40%|███▉      | 90/227 [00:01<00:02, 46.30it/s]







 42%|████▏     | 95/227 [00:01<00:02, 46.22it/s]

Epoch 19 | Step 4400 | Train_loss 3.138442277908325 | Val_loss = 3.1581051349639893










 44%|████▍     | 100/227 [00:01<00:02, 45.06it/s]







 47%|████▋     | 106/227 [00:02<00:02, 47.90it/s]







 50%|████▉     | 113/227 [00:02<00:02, 51.02it/s]







 52%|█████▏    | 119/227 [00:02<00:02, 47.32it/s]







 56%|█████▌    | 126/227 [00:02<00:01, 51.50it/s]







 58%|█████▊    | 132/227 [00:02<00:01, 53.39it/s]







 61%|██████    | 138/227 [00:02<00:01, 49.11it/s]







 63%|██████▎   | 144/227 [00:02<00:01, 51.17it/s]







 66%|██████▌   | 150/227 [00:02<00:01, 53.25it/s]







 69%|██████▉   | 157/227 [00:03<00:01, 55.36it/s]







 72%|███████▏  | 164/227 [00:03<00:01, 56.92it/s]







 75%|███████▍  | 170/227 [00:03<00:00, 57.24it/s]







 78%|███████▊  | 176/227 [00:03<00:01, 50.80it/s]







 80%|████████  | 182/227 [00:03<00:00, 52.34it/s]







 83%|████████▎ | 188/227 [00:03<00:00, 48.03it/s]







 85%|████████▌ | 193/227 [00:03<00:00, 45.28it/s]

Epoch 19 | Step 4500 | Train_loss 3.21132755279541 | Val_loss = 3.1368328332901










 88%|████████▊ | 199/227 [00:03<00:00, 48.14it/s]







 90%|█████████ | 205/227 [00:03<00:00, 50.60it/s]







 93%|█████████▎| 211/227 [00:04<00:00, 46.72it/s]







 96%|█████████▌| 217/227 [00:04<00:00, 48.87it/s]







Epoch: 100%|██████████| 20/20 [01:18<00:00,  4.29s/it]A






100%|██████████| 227/227 [00:23<00:00, 54.75it/s]





100%|██████████| 227/227 [00:20<00:00, 46.03it/s]







100%|██████████| 227/227 [00:15<00:00, 51.40it/s]

In [95]:
loss = criterion(prediction, trg)

In [96]:
loss.item()

8.685587882995605